# Conversion between astronomical coordinate systems

In [ ]:
import numpy as np

from _time import *
from _utils import *

### Helper functions

In [ ]:
class MatchingValueError(ValueError):
    """Exception raised when no matching values are found."""

## 1. Horizontal to Equatorial I

In [ ]:
lat, a, A = np.deg2rad(30), np.deg2rad(100), np.deg2rad(180)

lat = normalize_sym(lat, p=np.pi/2)
a = normalize_asym(a, p=np.pi)
A = (A + abs(a)//(np.pi/2) * np.pi) % (2*np.pi)

print(np.rad2deg(lat), np.rad2deg(a), np.rad2deg(A))

d = np.arcsin(
    np.sin(a) * np.sin(lat) + np.cos(a) * np.cos(lat) * np.cos(A))
d = normalize_sym(d, p=np.pi/2)

print(np.rad2deg(d))

In [ ]:
# Calculate Local Hour Angle (LHA and H)
# sin(H) = - sin(A) * cos(a) / cos(δ)
x = -np.sin(A) * np.cos(a) / np.cos(d)
Hs1 = np.arcsin(np.clip(x, -1, 1)) % (2*np.pi)

# The arcsin returns with exactly 1 value for H, but in this case
# it is ambigous, because the equation has another solution in the
# allowed interval. We evaluate it here.
Hs2 = (3*np.pi - Hs1) % (2*np.pi)

print(np.rad2deg(Hs1), np.rad2deg(Hs2))

In [ ]:
 # Now calculate LHA with a second method to determine which one is
# the actual solution out of H1 and H2
# cos(H) = (sin(a) - sin(δ) * sin(φ)) / (cos(δ) * cos(φ))
x = (np.sin(a) - np.sin(d) * np.sin(lat)) / (np.cos(d) * np.cos(lat))
Hc1 = np.arccos(np.clip(x, -1, 1)) % (2*np.pi)

# Again, the arccos returns with exactly 1 value for H, but it is
# again ambigous, because the equation has another solution in the
# allowed interval. We evaluate it here.
Hc2 = (2*np.pi - Hc1) % (2*np.pi)

print(np.rad2deg(Hc1), np.rad2deg(Hc2))

In [ ]:
def hor2equI(lat, a, A, LMST):
    '''
    Converts horizontal coordinates to geocentric equatorial coordinates.

    Parameters:
    -----------
    lat : float
        Latitude of the observer on Earth in radians.
    a : float
        The Altitude in radians.
    A : float
        The Azimuth in radians.
    LMST : float
        The Local Mean Sidereal Time in hours.

    Returns:
    --------
    t : float
        The Local Hour Angle in hours.
    RA : float
        The Right Ascension in hours.

    Notes:
    ------
    
    '''
    # Normalize input parameters to their expected ranges
    lat = normalize_sym(lat, p=np.pi/2)
    a = normalize_asym(a, p=np.pi)
    # If abs(Altitude) is >90˚, then flip Azimuth by 180˚!
    A = (A + abs(a)//(np.pi/2) * np.pi) % (2*np.pi)
    # Horizontal coordinates does not work on the poles!
    assert np.abs(lat) != np.pi/2, 'Poles are ambigous in horizontal coordinates!'

    # Calculate Declination (δ)
    # sin(δ) = sin(a) * sin(φ) + cos(a) * cos(φ) * cos(A)
    # The result for δ will be non-ambigous and the output of the arcsin
    # can be automatically accepted
    x = np.sin(a) * np.sin(lat) + np.cos(a) * np.cos(lat) * np.cos(A)
    d = normalize_sym(np.arcsin(np.clip(x, -1, 1)), p=np.pi/2)

    # Calculate Local Hour Angle (LHA and H)
    # sin(H) = - sin(A) * cos(a) / cos(δ)
    x = -np.sin(A) * np.cos(a) / np.cos(d)
    Hs1 = np.arcsin(np.clip(x, -1, 1)) % (2*np.pi)

    # The arcsin returns with exactly 1 value for H, but in this case
    # it is ambigous, because the equation has another solution in the
    # allowed interval. We evaluate it here.
    Hs2 = (3*np.pi - Hs1) % (2*np.pi)

    # Now calculate LHA with a second method to determine which one is
    # the actual solution out of H1 and H2
    # cos(H) = (sin(a) - sin(δ) * sin(φ)) / (cos(δ) * cos(φ))
    x = (np.sin(a) - np.sin(d) * np.sin(lat)) / (np.cos(d) * np.cos(lat))
    Hc1 = np.arccos(np.clip(x, -1, 1)) % (2*np.pi)

    # Again, the arccos returns with exactly 1 value for H, but it is
    # again ambigous, because the equation has another solution in the
    # allowed interval. We evaluate it here.
    Hc2 = (2*np.pi - Hc1) % (2*np.pi)

    # Select the correct H value by comparison of the various values
    if np.isclose(Hs1, Hc1) or np.isclose(Hs1, Hc2):
        H = Hs1
    elif np.isclose(Hs2, Hc1) or np.isclose(Hs2, Hc2):
        H = Hs2
    else:
        print(Hs1, Hc1, Hs2, Hc2)
        raise MatchingValueError("No matching value found among the " \
                                 "calculated Local Hour Angle values.")

    # Calculate Right Ascension (α)
    # α = S – t
    RA = (LMST - np.deg2rad(H)/15) % 24

    return d, RA

In [ ]:
hor2equI(lat=np.deg2rad(30), a=np.deg2rad(30), A=np.deg2rad(0), LMST=0)

### 2. Horizontal to Equatorial II

In [ ]:
def Hor_To_Equ_II(Latitude, Altitude, Azimuth, LMST=None):

    # First Convert Horizontal to Equatorial I Coordinates
    Coordinates = Hor_To_Equ_I(Latitude, Altitude, Azimuth, LMST)
    Declination = Coordinates[0]
    Right_Ascension = Coordinates[1]
    LHT = Coordinates[2]

    # Calculate LMST if it is not known
    if(LMST == None):
        LMST = LHT + Right_Ascension

    # Normalize LMST
    # LMST: [0,24h[
    LMST, _ = Normalize_Zero_Bounded(LMST, 24)

    Coordinates = np.array((Declination, Right_Ascension, LMST))
    return(Coordinates)

### 3. Equatorial I to Horizontal

In [ ]:
def Equ_I_To_Hor(Latitude,
                 Declination,
                 Right_Ascension,
                 LHT=None,
                 LMST=None,
                 Altitude=None):

    # Input data normalization
    # Latitude: [-π/2,+π/2]
    # Declination: [-π/2,+π/2]
    # Right Ascension: [0h,24h[
    Latitude = Normalize_Symmetrically_Bounded_PI_2(Latitude)
    Declination = Normalize_Symmetrically_Bounded_PI_2(Declination)
    Right_Ascension, _ = Normalize_Zero_Bounded(Right_Ascension, 24)
    
    # Accuracy for calculations
    accuracy = 0.00001

    # Calculate Local Hour Angle in Hours (t)
    if(LMST != None):
        # t = S - α
        LHT = LMST - Right_Ascension
        # Normalize LHT
        # LHT: [0h,24h[
        LHT, _ = Normalize_Zero_Bounded(LHT, 24)

    if(LHT != None):
        # Convert LHA to angles from hours (t -> H)
        LHA = LHT * 15

        # Calculate Altitude (m)
        # sin(m) = sin(δ) * sin(φ) + cos(δ) * cos(φ) * cos(H)
        # The result for m will be non-ambigous and the output of
        # the 'np.arcsin()' can be automatically accepted
        Altitude = np.degrees(np.arcsin(
                   np.sin(np.radians(Declination)) * np.sin(np.radians(Latitude)) +
                   np.cos(np.radians(Declination)) * np.cos(np.radians(Latitude)) *
                   np.cos(np.radians(LHA))
                   ))

        # Normalize result for Altitude: [-π/2,+π/2]
        Altitude = Normalize_Symmetrically_Bounded_PI_2(Altitude)

        # Calculate Azimuth (A)
        # sin(A) = - sin(H) * cos(δ) / cos(m)
        Azimuth_sin_1 = np.degrees(np.arcsin(
                      - np.sin(np.radians(LHA)) *
                        np.cos(np.radians(Declination)) /
                        np.cos(np.radians(Altitude))
                        ))

        # Normalize result for Azimuth: [0,+2π[
        Azimuth_sin_1, _ = Normalize_Zero_Bounded(Azimuth_sin_1, 360)

        # 'np.arcsin()' returns with exactly 1 value for A, but in this case
        # it is ambigous, because the equation has another solution in the 
        # correct interval. The second solution is evaluated below.
        Azimuth_sin_2 = 540 - Azimuth_sin_1

        # Calculate Azimuth (A) with a second method, to determine which one is the correct
        # cos(A) = (sin(δ) - sin(φ) * sin(m)) / (cos(φ) * cos(m))
        Azimuth_cos_1 = np.degrees(np.arccos(
                       (np.sin(np.radians(Declination)) - np.sin(np.radians(Latitude)) *
                        np.sin(np.radians(Altitude))) / 
                       (np.cos(np.radians(Latitude)) * np.cos(np.radians(Altitude)))
                        ))

        # 'np.arccos()' returns with exactly 1 value for A, but in this case
        # it is ambigous, because the equation has another solution in the 
        # correct interval. The second solution is evaluated below.
        Azimuth_cos_2 = 360 - Azimuth_cos_1

        # Normalize result for Azimuth: [0,+2π[
        Azimuth_cos_2, _ = Normalize_Zero_Bounded(Azimuth_cos_2, 360)

        # Compare Azimuth values
        if(np.abs(Azimuth_sin_1 - Azimuth_cos_1) < accuracy or
           np.abs(Azimuth_sin_1 - Azimuth_cos_2) < accuracy):

            Azimuth = Azimuth_sin_1

        elif(np.abs(Azimuth_sin_2 - Azimuth_cos_1) < accuracy or
             np.abs(Azimuth_sin_2 - Azimuth_cos_2) < accuracy):

            Azimuth = Azimuth_sin_2

        else:
            raise ValueError('The correct Azimuth value could not be estimated!')

        Coordinates = np.array((Altitude, Azimuth))
        return(Coordinates)

    elif(Altitude != None):
        
        # First check if the object is ever rise above the horizon,
        # or if it could exceed the given altitude
        Max_Altitude = 90 - (Declination - Latitude)
        
        if(Max_Altitude < 0):
            raise ValueError('Given object will never rise above the horizon!')
            
        if(Max_Altitude <= Altitude):
            raise ValueError('Given object will never rise above the given Altitude!')
        
        # Starting Equations: 
        # sin(m) = sin(δ) * sin(φ) + cos(δ) * cos(φ) * cos(H)
        # We can calculate eg. setting/rising with the available data (m = 0°), or other things...
        # First let's calculate LHA:
        # cos(H) = (sin(m) - sin(δ) * sin(φ)) / cos(δ) * cos(φ)
        LHA_1 = np.degrees(np.arccos(
              ((np.sin(np.radians(Altitude)) -
                np.sin(np.radians(Declination)) *
                np.sin(np.radians(Latitude))) /
               (np.cos(np.radians(Declination)) *
                np.cos(np.radians(Latitude))))))

        # 'np.arccos()' returns with exactly 1 value for H, but in this case
        # it is ambigous, because the equation has another solution in the 
        # correct interval. The second solution is evaluated below.
        LHA_2 = 360 - LHA_1

        # Normalize LHAs:
        LHA_1, _ = Normalize_Zero_Bounded(LHA_1, 360)
        LHA_2, _ = Normalize_Zero_Bounded(LHA_2, 360)

        #
        # Calculate Azimuth (A) for both Local Hour Angles!
        #
        # Calculate Azimuth (A) for the FIRST LHA
        # sin(A) = - sin(H) * cos(δ) / cos(m)
        Azimuth_sin_1 = np.degrees(np.arcsin(
                      - np.sin(np.radians(LHA_1)) *
                        np.cos(np.radians(Declination)) /
                        np.cos(np.radians(Altitude))
                        ))

        # Normalize result for Azimuth: [0,+2π[
        Azimuth_sin_1, _ = Normalize_Zero_Bounded(Azimuth_sin_1, 360)

        # 'np.arcsin()' returns with exactly 1 value for H, but in this case
        # it is ambigous, because the equation has another solution in the 
        # correct interval. The second solution is evaluated below.
        Azimuth_sin_2 = 540 - Azimuth_sin_1

        # Calculate Azimuth (A) with a second method, to determine which one is the correct
        # cos(A) = (sin(δ) - sin(φ) * sin(m)) / (cos(φ) * cos(m))
        Azimuth_cos_1 = np.degrees(np.arccos(
                       (np.sin(np.radians(Declination)) - np.sin(np.radians(Latitude)) *
                        np.sin(np.radians(Altitude))) / 
                       (np.cos(np.radians(Latitude)) * np.cos(np.radians(Altitude)))
                        ))

        # 'np.arccos()' returns with exactly 1 value for H, but in this case
        # it is ambigous, because the equation has another solution in the 
        # correct interval. The second solution is evaluated below.
        Azimuth_cos_2 = 360 - Azimuth_cos_1

        # Normalize result for Azimuth: [0,+2π[
        Azimuth_cos_2, _ = Normalize_Zero_Bounded(Azimuth_cos_2, 360)

        # Compare Azimuth values
        if(np.abs(Azimuth_sin_1 - Azimuth_cos_1) < accuracy or
           np.abs(Azimuth_sin_1 - Azimuth_cos_2) < accuracy):

            Azimuth_1 = Azimuth_sin_1

        elif(np.abs(Azimuth_sin_2 - Azimuth_cos_1) < accuracy or
             np.abs(Azimuth_sin_2 - Azimuth_cos_2) < accuracy):

            Azimuth_1 = Azimuth_sin_2

        else:
            raise ValueError('The correct Azimuth value could not be estimated!')

        # Calculate Azimuth (A) for the SECOND LHA
        # sin(A) = - sin(H) * cos(δ) / cos(m)
        Azimuth_sin_1 = np.degrees(np.arcsin(
                      - np.sin(np.radians(LHA_2)) *
                        np.cos(np.radians(Declination)) /
                        np.cos(np.radians(Altitude))
                        ))

        # Normalize result for Azimuth: [0,+2π[
        Azimuth_sin_1, _ = Normalize_Zero_Bounded(Azimuth_sin_1, 360)

        # 'np.arcsin()' returns with exactly 1 value for H, but in this case
        # it is ambigous, because the equation has another solution in the 
        # correct interval. The second solution is evaluated below.
        Azimuth_sin_2 = 540 - Azimuth_sin_1

        # Calculate Azimuth (A) with a second method, to determine which one is the correct
        # cos(A) = (sin(δ) - sin(φ) * sin(m)) / (cos(φ) * cos(m))
        Azimuth_cos_1 = np.degrees(np.arccos(
                       (np.sin(np.radians(Declination)) - np.sin(np.radians(Latitude)) *
                        np.sin(np.radians(Altitude))) / 
                       (np.cos(np.radians(Latitude)) * np.cos(np.radians(Altitude)))
                        ))

        # 'np.arccos()' returns with exactly 1 value for H, but in this case
        # it is ambigous, because the equation has another solution in the 
        # correct interval. The second solution is evaluated below.
        Azimuth_cos_2 = 360 - Azimuth_cos_1

        # Normalize result for Azimuth: [0,+2π[
        Azimuth_cos_2, _ = Normalize_Zero_Bounded(Azimuth_cos_2, 360)

        # Compare Azimuth values
        if(np.abs(Azimuth_sin_1 - Azimuth_cos_1) < accuracy or
           np.abs(Azimuth_sin_1 - Azimuth_cos_2) < accuracy):

            Azimuth_2 = Azimuth_sin_1

        elif(np.abs(Azimuth_sin_2 - Azimuth_cos_1) < accuracy or
             np.abs(Azimuth_sin_2 - Azimuth_cos_2) < accuracy):

            Azimuth_2 = Azimuth_sin_2

        else:
            raise ValueError('The correct Azimuth value could not be estimated!')

        # Calculate time between them
        # Use precalculated LHAs
        # H_dil is the time, where the Object stays below the given Altitude
        H_dil = np.abs(LHA_1 - LHA_2)

        Coordinates = np.array((Azimuth_1, Azimuth_2, H_dil))
        return(Coordinates)

    else:
        raise AttributeError('Either Altitude or LHT values must be given!')

### 4. Equatorial I to Equatorial II

In [ ]:
def Equ_I_To_Equ_II(Right_Ascension, t):
    
    LMST = t + Right_Ascension
    # Normalize LMST
    # LMST: [0,24h[
    LMST, _ = Normalize_Zero_Bounded(LMST, 24)

    Coordinates = np.array((Right_Ascension, LMST))
    return(Coordinates)

### 5. Equatorial II to Equatorial I

In [ ]:
def Equ_II_To_Equ_I(LMST,
                    Right_Ascension,
                    LHT):

    # Calculate Right Ascension or Local Mean Sidereal Time
    if(RightAscension != None and LHT == None):
        LHT = LMST - Right_Ascension

    elif(RightAscension == None and LHT != None):
        Right_Ascension = LMST - LHT

    else:
        raise AttributeError('Either Right Ascension or LHT values must be given!')
    
    # Normalize LHA
    # LHA: [0,24h[
    LHT, _ = Normalize_Zero_Bounded(LHT, 24)

    # Normalize Right Ascension
    # Right Ascension: [0,24h[
    Right_Ascension, _ = Normalize_Zero_Bounded(Right_Ascension, 24)

    Coordinates = np.array((Right_Ascension, LHT))
    return(Coordinates)

### 6. Equatorial II to Horizontal

In [ ]:
def Equ_II_To_Hor(Latitude,
                  Declination,
                  LMST,
                  Right_Ascension=None,
                  LHT=None):

    # Input data normalization
    # Latitude: [-π/2,+π/2]
    # Local Mean Sidereal Time: [0h,24h[
    # Local Hour Angle: [0h,24h[
    # Right Ascension: [0h,24h[
    # Declination: [-π/2,+π/2]
    Latitude = Normalize_Symmetrically_Bounded_PI_2(Latitude)
    LMST, _ = Normalize_Zero_Bounded(LMST, 24)
    
    if(Right_Ascension == None and LHT != None):
        LHT, _ = Normalize_Zero_Bounded(LHT, 24)

    elif(Right_Ascension != None and LHT == None):
        Right_Ascension, _ = Normalize_Zero_Bounded(Right_Ascension, 24)
        
    else:
        raise AttributeError('Either right ascension of LHT values must be given!')
    
    Declination = Normalize_Symmetrically_Bounded_PI_2(Declination)

    # Convert Equatorial II to Equatorial I
    Coordinates = Equ_II_To_Equ_I(LMST,
                                  Right_Ascension,
                                  LHT)
    Right_Ascension = Coordinates[0]
    LHT = Coordinates[1]

    # Convert Equatorial I to Horizontal
    Coordinates = Equ_I_To_Hor(Latitude,
                               Declination,
                               Right_Ascension,
                               LHT,
                               LMST,
                               Altitude)
    Altitude = Coordinates[0]
    Azimuth = Coordinates[1]

    # Output data normalization
    # Altitude: [-π/2,+π/2]
    # Azimuth: [0,+2π[
    Altitude = Normalize_Symmetrically_Bounded_PI_2(Altitude)
    Azimuth, _ = Normalize_Zero_Bounded(Azimuth, 360)

    Coordinates = np.array((Altitude, Azimuth))
    return(Coordinates)